# NEURAL NETWORKS AND DEEP LEARNING PROJECT

---
A.A. 2023/24 (6 CFU) - Giuseppe Labate, Sana Amiri
---

## Project 1 - Car classification, attribute prediction and car verification

In this project 

1.   **Classification** car images given cars dataset CompCars.

2.   **Attributes prediction** based on the images.  
        Attributes for this project are:  
        Explicit Attributes:  
                             a. Number of doors  
                             b. Number of seats  
                             c. Type of car  
        Implicit Attributes:  
                             d. Max speed  
                             e. Displacement  
